# Nelder-Mead Generator adapted from SciPy

Most of the algorithms in scipy.optimize are self-contained functions that operate on the user-provided `func`. Xopt has adapted the Nelder-Mead directly from scipy.optimize to be in a generator form. This allows for the manual stepping through the algorithm.


In [ ]:
from xopt.generators.sequential.neldermead import NelderMeadGenerator
from xopt import Evaluator, VOCS
from xopt.resources.test_functions.rosenbrock import rosenbrock

import pandas as pd

from xopt import Xopt
from xopt.vocs import random_inputs
import numpy as np

from scipy.optimize import fmin

# from xopt import output_notebook
# output_notebook()

import matplotlib.pyplot as plt

## Nelder-Mead optimization of the Rosenbrock function with Xopt

In [ ]:
YAML = """
max_evaluations: 500
generator:
  name: neldermead
  adaptive: true
evaluator:
  function: xopt.resources.test_functions.rosenbrock.evaluate_rosenbrock
vocs:
  variables:
    x0: [-5, 5]
    x1: [-5, 5]
  objectives: {y: MINIMIZE}
"""
X = Xopt.from_yaml(YAML)

In [ ]:
XMIN = [1, 1]  # True minimum

In [ ]:
X.random_evaluate(2)
X.run()
X.data

In [ ]:
# Evaluation progression
X.data["y"].plot()
plt.yscale("log")
plt.xlabel("iteration")
plt.ylabel("Rosenbrock value")

In [ ]:
# Minimum
dict(X.data.iloc[X.data["y"].argmin()])

## Visualize

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

Xgrid, Ygrid = np.meshgrid(np.linspace(-2, 2, 201), np.linspace(-2, 2, 201))

Zgrid = np.vectorize(lambda x, y: rosenbrock([x, y]))(Xgrid, Ygrid)
Zgrid = np.log(Zgrid + 1)

ax.pcolormesh(Xgrid, Ygrid, Zgrid)
ax.contour(Xgrid, Ygrid, Zgrid, levels=10, colors="black")
ax.set_xlabel("x0")
ax.set_ylabel("x1")


# Add all evaluations
ax.plot(X.data["x0"], X.data["x1"], color="red", alpha=0.5, marker=".")
ax.scatter(XMIN[0], XMIN[1], 50, marker="o", color="orange", label="True minimum")
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
# plt.legend()
ax.set_title("Xopt's Nelder-Mead progression")

In [ ]:
# Manually step the algorithm and collect simplexes
X = Xopt.from_yaml(YAML)
X.random_evaluate(1)
simplexes = []
for i in range(500):
    X.step()
    simplexes.append(X.generator.simplex)

In [ ]:
def plot_simplex(simplex, ax=None):
    x0 = simplex["x0"]
    x1 = simplex["x1"]
    x0 = np.append(x0, x0[0])
    x1 = np.append(x1, x1[0])
    ax.plot(x0, x1)


fig, ax = plt.subplots(figsize=(8, 8))
ax.pcolormesh(Xgrid, Ygrid, Zgrid)
# ax.contour(Xgrid, Ygrid, Zgrid, levels=10, colors='black')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_xlabel("x0")
ax.set_ylabel("x1")
ax.set_title("Nelder-Mead simplex progression")

ax.scatter(XMIN[0], XMIN[1], 50, marker="o", color="orange", label="True minimum")

for simplex in simplexes:
    plot_simplex(simplex, ax)

## Compare with scipy.optimize.fmin Nelder-Mead

Notice that fmin is much faster here. This is because the function runs very fast, so the internal Xopt bookkeeping overhead dominates.


In [ ]:
result = fmin(rosenbrock, [-1, -1])
result

In [ ]:
X = Xopt.from_yaml(YAML)
X.random_evaluate(1)

In [ ]:
X.run()
# Almost exactly the same number evaluations.
len(X.data)

In [ ]:
# results are the same
xbest = X.data.iloc[X.data["y"].argmin()]
xbest["x0"] == result[0], xbest["x1"] == result[1]

# NelderMeadGenerator object

In [ ]:
NelderMeadGenerator.model_fields

In [ ]:
Xbest = [33, 44]


def f(inputs, verbose=False):
    if verbose:
        print(f"evaluate f({inputs})")
    x0 = inputs["x0"]
    x1 = inputs["x1"]

    # if x0 < 10:
    #    raise ValueError('test XXXX')

    y = (x0 - Xbest[0]) ** 2 + (x1 - Xbest[1]) ** 2

    return {"y": y}


ev = Evaluator(function=f)
vocs = VOCS(
    variables={"x0": [-100, 100], "x1": [-100, 100]}, objectives={"y": "MINIMIZE"}
)
vocs.json()

In [ ]:
# check output
f(random_inputs(vocs)[0])

In [ ]:
G = NelderMeadGenerator(vocs=vocs, initial_point={"x0": 0, "x1": 0})
inputs = G.generate(1)
inputs

In [ ]:
# Further generate calls will continue to produce same point, as with BO
G.generate(1)

In [ ]:
ev.evaluate(inputs[0])

In [ ]:
# Adding new data will advance state to next step, and next generate() will yield new point
G.add_data(pd.DataFrame(inputs[0] | ev.evaluate(inputs[0]), index=[0]))
G.generate(1)

In [ ]:
# Create Xopt object
X = Xopt(
    evaluator=ev,
    vocs=vocs,
    generator=NelderMeadGenerator(vocs=vocs),
    max_evaluations=100,
)

# Optional: give an initial pioint
X.generator.initial_point = {"x0": 0, "x1": 0}

In [ ]:
X.run()

In [ ]:
# This shows the latest simplex
X.generator.simplex

In [ ]:
X.data["y"].plot()
plt.yscale("log")

In [ ]:
fig, ax = plt.subplots()
X.data.plot("x0", "x1", ax=ax, color="black", alpha=0.5)
ax.scatter(Xbest[0], Xbest[1], marker="x", color="red")

In [ ]:
# This is the raw internal state of the generator
a = X.generator.current_state
a

## 5-dimensional Rosenbrock

`evaluate_rosenbrock` works for arbitrary dimensions, so adding more variables to `vocs` transforms this problem.

In [ ]:
YAML = """
max_evaluations: 500
generator:
  name: neldermead
evaluator:
  function: xopt.resources.test_functions.rosenbrock.evaluate_rosenbrock
vocs:
  variables:
    x1: [-5, 5]
    x2: [-5, 5]
    x3: [-5, 5]
    x4: [-5, 5]
    x5: [-5, 5]
  objectives:
    y: MINIMIZE
"""
X = Xopt.from_yaml(YAML)

In [ ]:
X.random_evaluate(1)
X.run()
X.data["y"].plot()
plt.yscale("log")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

Xgrid, Ygrid = np.meshgrid(np.linspace(-2, 2, 201), np.linspace(-2, 2, 201))

Zgrid = np.vectorize(lambda x, y: rosenbrock([x, y, 1, 1, 1]))(
    Xgrid, Ygrid
)  # The minimum is at 1,1,1,1,1
Zgrid = np.log(Zgrid + 1)

ax.pcolormesh(Xgrid, Ygrid, Zgrid)
ax.contour(Xgrid, Ygrid, Zgrid, levels=10, colors="black")
ax.set_xlabel("x0")
ax.set_ylabel("x1")


# Add all evaluations
ax.plot(X.data["x1"], X.data["x2"], color="red", alpha=0.5, marker=".")
ax.scatter(XMIN[0], XMIN[1], 50, marker="o", color="orange", label="True minimum")
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
# plt.legend()
ax.set_title("Xopt's Nelder-Mead progression")